## Day 36 Proyect: Stock News Notifier

### Stock News Notifier 📈📰 – Proyecto del Día 36

Este proyecto te alerta cuando el precio de una acción (por ejemplo, Tesla - TSLA) cambia significativamente.  
Cuando ocurre una variación importante, se consulta la API de noticias y se envía un resumen por WhatsApp.

📌 Se integran habilidades como:
- Consumo de APIs (Alpha Vantage y NewsAPI)
- Análisis de cambios porcentuales
- Slicing de listas
- Formato condicional con emojis 📊
- Automatización de notificaciones vía WhatsApp (Twilio)


#### Paso 1: Consulta del precio de acciones con Alpha Vantage

Obtenemos los precios de cierre de los dos últimos días para calcular la diferencia porcentual.


In [1]:
import requests
import os
from twilio.rest import Client

In [ ]:
# Parametros para consultar la API de Alpha Vantage
STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"
ALPHA_VANTAGE_API_KEY = "ALPHA_VANTAGE_API_KEY"
stock_url = "https://www.alphavantage.co/query"


stock_params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK,
    "apikey": ALPHA_VANTAGE_API_KEY
}


In [3]:
# Crear la solicitud al API de Alpha Vantage
response = requests.get(url=stock_url
                        ,params=stock_params
                        )

# Validar la solicitud
if response.status_code == 200:                         #Si es exitosa imprimimos el mensaje de éxito
    print("✅ Solicitud exitosa")
else:
    print("❌ Error:", response.status_code)            # Si no es exitosa imprimimos el mensaje de error
    print("Respuesta:", response.json()["message"]) 

✅ Solicitud exitosa


In [4]:
# Obtener el diccionario de datos por día
data = response.json()["Time Series (Daily)"]

# Convertir a lista ordenada
data_list = [value for (date, value) in data.items()]

# Obtener los precios de cierre de ayer y anteayer
yesterday_close = float(data_list[0]["4. close"])               # Convertir a float para poder hacer operaciones matemáticas
day_before_yesterday_close = float(data_list[1]["4. close"])    # Convertir a float para poder hacer operaciones matemáticas

# Calcular diferencia
difference = yesterday_close - day_before_yesterday_close

# Calcular porcentaje de diferencia
diff_percent = round(abs(difference) / day_before_yesterday_close * 100)

# Asignar el emoji de subida o bajada
up_down = "🔼" if difference > 0 else "🔻"

# Mostrar el resultado de manera amigable
print(f"{STOCK}: {up_down} {diff_percent}%")

TSLA: 🔻 2%


#### Paso 2: Si hay cambio importante, consultar noticias relevantes 🗞️

Usamos NewsAPI para buscar noticias recientes que mencionen el nombre de la compañía.


In [ ]:
# Parametros para consultar la API de News API
#NEWS_API_KEY = os.environ.get("NEWS_API_KEY")
NEWS_API_KEY = "NEWS_API_KEY"
news_url = "https://newsapi.org/v2/everything"
news_params = {
    "qInTitle": COMPANY_NAME,
    "apiKey": NEWS_API_KEY,
    "language": "en"
}

In [6]:
# Consutar la API de NewsAPI si la diferencia es mayor al 1%
if diff_percent > 1:
  # Crear la solicitud al API de NewsAPI
  news_response = requests.get(url=news_url
                              , params=news_params)
  
  # Validar la solicitud
  if news_response.status_code == 200:                       # Si es exitosa imprimimos el mensaje de éxito
    print("✅ Solicitud exitosa")
  else:
    print("❌ Error:", news_response.status_code)            # Si no es exitosa imprimimos el mensaje de error
    print("Respuesta:", news_response.json()["message"]) 

# Obtener el diccionario de datos de noticias
  articles = news_response.json()["articles"]

  # Seleccionar 3 noticias principales
  top_3_articles = articles[:3]

  # Formatear artículos
  formatted_articles = [
      f"{STOCK}: {up_down}{diff_percent}%\nHeadline: {article['title']}\nDate: {article['publishedAt']}\nBrief: {article['description']}"
      #f"Headline: {article['title']}.\nBrief: {article['description']}"
      for article in top_3_articles
  ]

  print(f"Artículos formateados: {formatted_articles}")


# Si la diferencia es menor a 1% no se consulta la API de NewsAPI
else:
  print("La diferencia es menor al 5%")


✅ Solicitud exitosa
Artículos formateados: ['TSLA: 🔻2%\nHeadline: Analyst Report: Tesla Inc\nDate: 2025-04-24T10:58:25Z\nBrief: None', 'TSLA: 🔻2%\nHeadline: Tesla, Inc. (TSLA): A Bull Case Theory\nDate: 2025-04-28T16:33:52Z\nBrief: None', 'TSLA: 🔻2%\nHeadline: Tesla, Inc. (TSLA): A Bull Case Theory\nDate: 2025-04-08T13:17:04Z\nBrief: None']


#### Paso 3: Enviar resumen por WhatsApp con Twilio 📲

Se envía cada artículo como un mensaje separado al número verificado por el sandbox.


In [ ]:
# Twilio credentials 
# TWILIO_SID = os.getenv("TWILIO_SID")
# TWILIO_AUTH_TOKEN = os.environ.get("TWILIO_AUTH_TOKEN")

TWILIO_SID ="TWILIO_SID"
TWILIO_AUTH_TOKEN = "TWILIO_SID"


# Sandbox de WhatsApp
WHATSAPP_FROM = "whatsapp:+1415523xxxx"   # Número de Twilio Sandbox
WHATSAPP_TO = "whatsapp:+521556696xxxx"   # Tu número verificado con prefijo

In [8]:
client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

for article in formatted_articles:
    message = client.messages.create(
        from_=WHATSAPP_FROM,  # Twilio Sandbox
        to=WHATSAPP_TO,     # Número verificado
        body=article
    )
    print(f"✅ Mensaje enviado: {message.body}")


✅ Mensaje enviado: TSLA: 🔻2%
Headline: Analyst Report: Tesla Inc
Date: 2025-04-24T10:58:25Z
Brief: None
✅ Mensaje enviado: TSLA: 🔻2%
Headline: Tesla, Inc. (TSLA): A Bull Case Theory
Date: 2025-04-28T16:33:52Z
Brief: None
✅ Mensaje enviado: TSLA: 🔻2%
Headline: Tesla, Inc. (TSLA): A Bull Case Theory
Date: 2025-04-08T13:17:04Z
Brief: None


#### 🧠 Conclusión

Este proyecto combinó múltiples componentes del mundo real:

- 📊 Consultar datos históricos de acciones con Alpha Vantage
- 📉 Analizar cambios porcentuales significativos
- 📰 Buscar noticias relacionadas con empresas usando NewsAPI
- 💬 Automatizar notificaciones a través de WhatsApp

Este flujo puede escalarse para alertas financieras, sistemas de monitoreo bursátil o trading algorítmico.
